## Creación de una base de datos unificada.

<img src="../img/database_schema.png" alt="Estructura de la base de datos" width="70%">


In [2]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

Creo el motor de base de datos.

In [5]:
engine = create_engine('sqlite:///../database.db', echo=True)

In [11]:
def crear_tabla(nombre, data: pd.DataFrame):
    try:
        data.to_sql(name=nombre, con=engine, index=False, if_exists='replace')
        print(f'Tabla "{nombre}" creada con exito.')
    except:
        raise "Error al crear la tabla."

### Tabla `clima`.

In [6]:
df = pd.read_parquet('../data/datos_climaticos.parquet')

In [12]:
crear_tabla('clima', df)

2025-10-23 18:04:09,132 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-23 18:04:09,149 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clima")
2025-10-23 18:04:09,150 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,154 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clima")
2025-10-23 18:04:09,155 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,157 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-23 18:04:09,158 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,160 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-10-23 18:04:09,162 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:09,164 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("clima")
2025-10-23 18:04:09,165 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:0

### Tabla `contagios`

In [10]:
df2 = pd.read_csv('../data/dengue-final.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68126 entries, 0 to 68125
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_uta                   68126 non-null  int64  
 1   departamento_nombre      68126 non-null  object 
 2   provincia_nombre         68126 non-null  object 
 3   ano                      68126 non-null  int64  
 4   semanas_epidemiologicas  68126 non-null  float64
 5   grupo_edad_id            68126 non-null  int64  
 6   grupo_edad_desc          68126 non-null  object 
 7   cantidad_casos           68126 non-null  float64
 8   poblacion                68126 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 4.7+ MB


In [13]:
crear_tabla('cotagios', df2)

2025-10-23 18:04:42,245 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-23 18:04:42,254 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cotagios")
2025-10-23 18:04:42,255 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:42,256 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cotagios")
2025-10-23 18:04:42,258 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:04:42,264 INFO sqlalchemy.engine.Engine 
CREATE TABLE cotagios (
	id_uta BIGINT, 
	departamento_nombre TEXT, 
	provincia_nombre TEXT, 
	ano BIGINT, 
	semanas_epidemiologicas FLOAT, 
	grupo_edad_id BIGINT, 
	grupo_edad_desc TEXT, 
	cantidad_casos FLOAT, 
	poblacion BIGINT
)


2025-10-23 18:04:42,265 INFO sqlalchemy.engine.Engine [no key 0.00115s] ()
2025-10-23 18:04:42,981 INFO sqlalchemy.engine.Engine INSERT INTO cotagios (id_uta, departamento_nombre, provincia_nombre, ano, semanas_epidemiologicas, grupo_edad_id, grupo_edad_desc, cantidad_casos, poblacion) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-

### Tabla `calendario`.

In [14]:
df3 = pd.read_csv('../data/calendario.csv')
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24004 entries, 0 to 24003
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fecha            24004 non-null  object
 1   anio             24004 non-null  int64 
 2   mes              24004 non-null  int64 
 3   dia              24004 non-null  int64 
 4   trimestre        24004 non-null  int64 
 5   semestre         24004 non-null  int64 
 6   quincena         24004 non-null  int64 
 7   semanaMes        24004 non-null  int64 
 8   semana           24004 non-null  int64 
 9   diaSemana        24004 non-null  object
 10  diaNumeroSemana  24004 non-null  int64 
 11  bisiesto         24004 non-null  bool  
dtypes: bool(1), int64(9), object(2)
memory usage: 2.0+ MB


In [15]:
crear_tabla('calendario', df3)

2025-10-23 18:05:49,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-23 18:05:49,806 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("calendario")
2025-10-23 18:05:49,807 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:05:49,809 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("calendario")
2025-10-23 18:05:49,810 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:05:49,816 INFO sqlalchemy.engine.Engine 
CREATE TABLE calendario (
	fecha TEXT, 
	anio BIGINT, 
	mes BIGINT, 
	dia BIGINT, 
	trimestre BIGINT, 
	semestre BIGINT, 
	quincena BIGINT, 
	"semanaMes" BIGINT, 
	semana BIGINT, 
	"diaSemana" TEXT, 
	"diaNumeroSemana" BIGINT, 
	bisiesto BOOLEAN
)


2025-10-23 18:05:49,818 INFO sqlalchemy.engine.Engine [no key 0.00272s] ()
2025-10-23 18:05:50,162 INFO sqlalchemy.engine.Engine INSERT INTO calendario (fecha, anio, mes, dia, trimestre, semestre, quincena, "semanaMes", semana, "diaSemana", "diaNumeroSemana", bisiesto) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 

### Tabla `ubicacion`

In [18]:
df_estaciones = pd.read_csv('../data/estaciones-meteorologicas-inta.csv')

In [34]:
lista_provincias = df2.provincia_nombre.apply(lambda x: ' '.join(p.capitalize() for p in x.split(' '))).unique()
print(lista_provincias)

['Ciudad De Buenos Aires' 'Buenos Aires' 'Mendoza' 'Santa Fe' 'Rio Negro'
 'Cordoba' 'Corrientes' 'Formosa' 'Chaco' 'La Rioja' 'San Luis'
 'Entre Rios' 'Misiones' 'Salta' 'Santiago Del Estero' 'Tucuman' 'Jujuy'
 'Catamarca' 'Neuquen' 'Chubut' 'La Pampa' 'San Juan' 'Santa Cruz'
 'Tierra Del Fuego Antartida E Islas Del Atlantico Sur']


In [39]:
df_provincias = pd.DataFrame({'provincia': lista_provincias})
df_provincias

,provincia
0,Ciudad De Buenos Aires
1,Buenos Aires
2,Mendoza
3,Santa Fe
4,Rio Negro
5,Cordoba
6,Corrientes
7,Formosa
8,Chaco
9,La Rioja


In [48]:
mapeo_renombrar = {
    'Cordoba': 'Córdoba', 
    'Tierra Del Fuego Antartida E Islas Del Atlantico Sur': 'Tierra del Fuego',
    'Ciudad De Buenos Aires': 'CABA',
    'Tucuman': 'Tucumán',
    'Neuquen': 'Neuquén',
    'Entre Rios': 'Entre Ríos',
    'Rio Negro': 'Río Negro'
}

df_provincias = df_provincias.replace(mapeo_renombrar).sort_values('provincia').reset_index(drop=True)
df_provincias

,provincia
0,Buenos Aires
1,CABA
2,Catamarca
3,Chaco
4,Chubut
5,Corrientes
6,Córdoba
7,Entre Ríos
8,Formosa
9,Jujuy


In [49]:
df_provincias['id'] = range(len(df_provincias))
df_provincias

,provincia,id
0,Buenos Aires,0
1,CABA,1
2,Catamarca,2
3,Chaco,3
4,Chubut,4
5,Corrientes,5
6,Córdoba,6
7,Entre Ríos,7
8,Formosa,8
9,Jujuy,9


In [50]:
crear_tabla('provincias', df_provincias)

2025-10-23 18:46:56,135 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-23 18:46:56,140 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("provincias")
2025-10-23 18:46:56,141 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:46:56,143 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("provincias")
2025-10-23 18:46:56,144 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-23 18:46:56,146 INFO sqlalchemy.engine.Engine 
CREATE TABLE provincias (
	provincia TEXT, 
	id BIGINT
)


2025-10-23 18:46:56,148 INFO sqlalchemy.engine.Engine [no key 0.00145s] ()
2025-10-23 18:46:56,157 INFO sqlalchemy.engine.Engine INSERT INTO provincias (provincia, id) VALUES (?, ?)
2025-10-23 18:46:56,160 INFO sqlalchemy.engine.Engine [generated in 0.00292s] [('Buenos Aires', 0), ('CABA', 1), ('Catamarca', 2), ('Chaco', 3), ('Chubut', 4), ('Corrientes', 5), ('Córdoba', 6), ('Entre Ríos', 7)  ... displaying 10 of 24 total bound parameter sets ...  ('Tierra del Fuego', 22), ('Tucumán', 23)]
2025

### Tabla `localidades`

In [51]:
df_estaciones

,Id,Nombre,Tipo,Localidad,Provincia,Latitud,Longitud,Altura,Id Interno,Ubicacion,Desde,Hasta
0,413,25 de Mayo - EEA Pergamino,Nimbus THP,25 de Mayo,Buenos Aires,-35.48,-60.13,85,A872872,CC 18 CP6660,04/09/2012,13/09/2025
1,682,5000,Nimbus THP,Hurlingham,Buenos Aires,-85.00,-55.00,0,A875000,-,20/11/2019,13/09/2025
2,683,5001,Nimbus THP,Hurlingham,Buenos Aires,-85.00,-60.00,0,A875001,-,20/11/2019,13/09/2025
3,704,993 - LAB,Nimbus THP,loc_prueba,Sin asignar,-84.00,-70.00,0,A872993,LABORATORIO,08/03/2022,26/08/2024
4,705,994 - LAB,Nimbus THP,loc_prueba,Sin asignar,-84.00,-65.00,0,A872994,LABORATORIO,01/01/2012,28/08/2025
...,...,...,...,...,...,...,...,...,...,...,...,...
164,477,Villa Dolores - EEA Manfredi,Nimbus THP,Manfredi,Córdoba,-31.94,-65.22,7.077.000.122.070.310,A872826,-,21/08/2015,13/09/2025
165,420,Villa Mercedes - EEA San Luis,Nimbus THP,Villa Mercedes,San Luis,-33.66,-65.41,515,A872930,"Rutas 7 Km 591, CP: 5730",19/08/2014,25/04/2024
166,694,Villa Paranacito-EEA C. del Ur,Nimbus THP,Villa Paranacito,Entre Rios,-33.71,-58.65,10,A872804,"Arroyo La Tinta, Villa Paranacito. C.P.: 2823",12/04/2021,13/09/2025
167,695,Villa Ramallo - EEA San Pedro,Nimbus THP,Villa Ramallo,Buenos Aires,-33.52,-60.11,10,A872948,"Ruta 9, km 208, Villa Ramallo",20/05/2021,20/07/2025


## Chequeo de los datos en *`database.db`*

In [5]:
conn = sqlite3.connect('base_datos.db')
cursor = conn.cursor()

In [7]:
cursor.execute('SELECT * FROM clima LIMIT 5;').fetchall()

[('2017-09-19 00:00:00.000000',
  'A872801',
  0.0,
  10.9,
  23.0,
  17.3875,
  74.0,
  14.14647,
  16.19621,
  19.46478,
  9.991712,
  83.0),
 ('2017-09-20 00:00:00.000000',
  'A872801',
  0.4957418023473161,
  15.9,
  27.1,
  20.12014,
  70.0,
  15.29874,
  17.66667,
  9.588419,
  2.886585,
  24.0),
 ('2017-09-21 00:00:00.000000',
  'A872801',
  0.5260351212713814,
  9.4,
  16.6,
  11.65834,
  98.0,
  9.864174,
  12.19869,
  1.33866,
  6.4341365,
  53.5),
 ('2017-09-22 00:00:00.000000',
  'A872801',
  0.0,
  4.7,
  19.6,
  11.72778,
  75.0,
  7.075937,
  10.12698,
  19.80138,
  9.981688,
  83.0),
 ('2017-09-23 00:00:00.000000',
  'A872801',
  0.0,
  5.0,
  23.8,
  13.49166,
  65.0,
  8.476982,
  11.17051,
  19.09344,
  9.404964,
  78.0)]